The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

``` bash
#!/bin/bash

cd $HOME
if [ ! -d "students" ]; then
    mkdir students
else
    echo "Directory students already existing."
fi
if [ -f "students/LCP_22-23_students.csv" ]; then
    echo "To-be-downloaded file already existing."
else
    wget -O "students/LCP_22-23_students.csv" -q "https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv"
fi
#
# 1.2: copy the students in two separate files, for Physics and PoD.
if [ -f "students/pod.csv" ]; then
    echo "PoD file already existing."
else
    grep "PoD" "students/LCP_22-23_students.csv" > "students/pod.csv"
fi
if [ -f "students/physics.csv" ]; then
    echo "Physics file already existing."
else
    grep "Physics" "students/LCP_22-23_students.csv" > "students/physics.csv"
fi
#1.3
```

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

``` bash
max=0
max_letter="a"
for letter in {a..z}; do
    count=$(grep -i "^$letter" "students/LCP_22-23_students.csv" | wc -l)
    echo "$letter: $count"
    if [ "$count" -gt "$max" ]; then
        max=$count #update new maximum
        max_letter=$letter
    fi
done
echo "Letter with max counts: $max_letter"


```

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

``` bash
for group in {1..18}; do
    grep -n "^" "students/LCP_22-23_students.csv" | awk -F: -v g="$group" '($1 - 1) % 18 + 1 == g { print $2 }' > "students/group_${group}.csv"
done
```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

```bash 
cd $HOME/test
if [ -f "data.txt" ]; then
    echo "Extracted file already existing."
else
    grep -v '^#' "data.csv" | tr -d ',' > "data.txt"
fi
```

2\.b How many even numbers are there?

```bash 
even_count=$(grep -o '[0-9]\+' 'data.txt' | awk '$1 % 2 == 0' | wc -l)
echo "Even numbers: $even_count"

```

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

```bash 
threshold=$(echo "100 * sqrt(3) / 2" | bc -l)
count_greater=0
count_smaller=0
while read -r x y z _ _ _; do
    value=$(echo "sqrt($x^2 + $y^2 + $z^2)" | bc -l)
    if (( $(echo "$value > $threshold" | bc -l) )); then
        ((count_greater++))
    else
        ((count_smaller++))
    fi
done < "data.txt"
echo "Smaller numbers: $count_smaller"
echo "Larger numbers: $count_greater"
```

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash 
if [ -z "$1" ]; then
    echo "value for n?"
    exit 1
fi
n=$1
mkdir -p "copies"
for ((i=1; i<=n; i++)); do
    awk -v divisor="$i" '{ for (j=1; j<=NF; j++) $j = $j / divisor }1' "data.txt" > "copies/data_copy_$i.txt"
done

```